# Data Visualisation Assignment

Total points: 80 + 20 (5 x 4) for conciseness

Total number of questions: 3 (45 + 30 + 25)

Dataset: [Titanic](https://www.kaggle.com/competitions/titanic)


In this assignment, we'll be using the help of data visualization to help us deal with missing values. Normally, you would think of a method to fill the `NaN` values, visualize it graphically to see if  your method is feasable, and then implement it.

Here however, since this will be the first time many people are doing this, I'll be giving a hint to the method, from which you should figure it out, implement it and then visualize only to justify your method.


**Note:** for every subquestion, use a copy of the `titanic` dataframe and not the original.

##  Data Dictionary:

- `PassengerId`, `Name`, `Sex`, `Age` and `Fare` are self-explanatory
- `Survived` is the variable of whether the passenger survived or not:
  - `1`: Survived
  - `0`: Didn't survive
- `Pclass` (Passenger Class) is the ticket class booked  by the passenger and it reflects the socio-economic status of the passenger:
  - `1`: Upper Class
  - `2`: Middle Class
  - `3`: Lower Class
- `SibSp` is the total number of the passengers' siblings and spouse aboard the ship
- `Parch` is the total number of the passengers' parents and children aboard the ship
- `Ticket` is the ticket number of the passenger
- `Cabin` is the cabin number of the passenger
- `Embarked` is port of embarkation (boarding):
  - `C`: Cherbourg
  - `Q`: Queenstown
  - `S`: Southampton


In [1]:
# imports 

import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")


In [2]:
titanic = pd.read_csv(r"D:\GitHub\crux-ml-summer-group-2022\2_Data_Vizualization\titanic.csv")
titanic.head()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\GitHub\\crux-ml-summer-group-2022\\2_Data_Vizualization\\titanic.csv'

### Dealing with missing values

We first find out which columns have missing values, and then deal with them one by one

In [ ]:
# function to help you keep track of missing values in a dataframe
def count_na(df):
  series = df.isna().sum()
  return series[series > 0]

In [ ]:
count_na(titanic)

Age         177
Cabin       687
Embarked      2
dtype: int64

#### Embarked

we start with embarked as it has only 2 missing values.

In [ ]:
titanic[titanic["Embarked"].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


- **Method 1:** We find out where place most people embarked from and fill in the missing values with the most common one.

This makes sense, but how would we show statistical evidence in a report?

----

Q1.1. Write a function that given a dataframe and column, visualizes the absolute and relative frequencies of the various values in the column (not including `NaN`) (10)

Figure shoud be made up of two subplots. (optional if you can't do it) (2)

Using this function, visualize and find out the confidence with which you can fill in the missing values in the `Embarked` column with its mode. (8)

----


In [ ]:
df = titanic.copy()
emb = df[~df["Embarked"].isna()]["Embarked"].mode()[0]
df["Embarked"].fillna(value = emb, inplace = True)



def frequency_plot(column, main_df):


    rel_freq = main_df[column].value_counts(normalize=True)*100
    freq = main_df[column].value_counts()

    x = rel_freq.index
    
    # layout
    fig = make_subplots(
        rows = 1, 
        cols = 2,
        subplot_titles = ("Frequency", "Relative Frequency")
    ) 

    # trace for frequency
    fig.add_trace(
        go.Bar(
            y = freq, 
            x = x,
            text = freq,
            hoverinfo = "x+y",
            name = "frequency"),
            row=1,
            col=1
        )

    # trace for relative frequency
    fig.add_trace(
        go.Bar(
            y = rel_freq, 
            x = x,
            text = rel_freq.round(2),
            hoverinfo = "x+y", 
            name = "relative frequency"),
            row=1,
            col=2
        )


    fig.layout.update(
        hovermode = "x",
        showlegend = True,
        )
    
    return fig

In [ ]:
frequency_plot("Embarked", titanic)

Thus we can say with ~72% confidence that the missing values in the `Embarked` column can be filled in with the mode, which is `S`. Since it high enough, the choice is justified.

----

Q1.2. Find another method to fill in the missing values in the `Embarked` column. (10)

Hint: When you book tickets for a train/plane with your friends together, you get assigned berths/seats next to each other/nearby.

In this question, consider *nearby* to be +- 5 cabins in the same section of the ship.

Again, justify the confidence of your answer, with the help of the function defined in 1.1 (5)

----

In [ ]:
df = titanic.copy()
df[df["Embarked"].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [ ]:
print(df["Cabin"][~df["Cabin"].isna()].head(10), end="\n\n")

1             C85
3            C123
6             E46
10             G6
11           C103
21            D56
23             A6
27    C23 C25 C27
31            B78
52            D33
Name: Cabin, dtype: object



We should check for cabins close to B28.

Since some values of cabins are lists, we need to check if we should consider them or not.

In [ ]:
s = df["Cabin"]
s = s[~s.isna()]

for i in s:
    # space separated values here form the list of cabins
    if " " in str(i) :
        if "B" in str(i):
            print(i)

B58 B60
B58 B60
B57 B59 B63 B66
B96 B98
B96 B98
B51 B53 B55
B57 B59 B63 B66
B96 B98
B82 B84
B96 B98
B51 B53 B55


In [ ]:
df = df.dropna(subset=["Cabin"])
df = df[df["Cabin"].str.contains("B")]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,1,0,PC 17569,146.5208,B78,C
54,55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0000,B28,NaN
118,119,0,1,"Baxter, Mr. Quigg Edmond",male,24.0,0,1,PC 17558,247.5208,B58 B60,C
139,140,0,1,"Giglio, Mr. Victor",male,24.0,0,0,PC 17593,79.2000,B86,C


In [ ]:
# since we ahve only B values, dropping the letter
df["Cabin"] = [[int(c[1:]) for c in x.split()] for x in df["Cabin"]]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,1,0,PC 17569,146.5208,[78],C
54,55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,[30],C
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0000,[28],NaN
118,119,0,1,"Baxter, Mr. Quigg Edmond",male,24.0,0,1,PC 17558,247.5208,"[58, 60]",C
139,140,0,1,"Giglio, Mr. Victor",male,24.0,0,0,PC 17593,79.2000,[86],C


In [ ]:
df = df[["Name", "Cabin", "Embarked"]]
df.reset_index(drop = True, inplace = True)

ndf = pd.DataFrame(columns = ["Name", "Cabin", "Embarked"])
_range = list(range(23, 34))


for i in range(len(df)):
    # if our list doesnt conatin any coomon cabin in the given range
    if set(df["Cabin"][i]).intersection(set(_range)) == set():
        continue
    else:
        ndf = ndf.append(df.iloc[i])

frequency_plot("Embarked", ndf)

In [ ]:
ndf

,Name,Cabin,Embarked
1,"Ostby, Mr. Engelhart Cornelius",[30],C
2,"Icard, Miss. Amelie",[28],NaN
44,"Stone, Mrs. George Nelson (Martha Evelyn)",[28],NaN


NOTE: From this method, though we apparently can say with ~100% cofindence that the choice should be `C`, we have only one datapoint. So in this case, Method 1 would be better, however you could come across a dataset that favours this method. (just wanted to show you that only your imagination is the limit for finding ways to fill `NaN` values :p, and your final choice depends solely on the data you are working with)

----

#### Cabin

The majority of the `Cabin` column is `NaN`. Does that mean we just drop the column?
We can, but we'll be losing data. Since we'll be making most of the data here artificially (i.e., not collected etc.), we need a very strong reason when filling these values.

----

Q2. Find a method to fill in the missing values in the `Cabin` column. (15)

Hint: It is impossible to to find/fill exactly where the passenger resided in on the ship. But if we drop the numbers in the cabin and retain only the letter, we now can classify the passengers into sections of the ship. Which ***categorical*** column  from our dataframe would be best suited for classifying passengers into different sections of the ship?

Visualize the density of the various categories in each section of the ship, using  
[stacked bar charts](https://study.com/academy/lesson/what-is-a-stacked-bar-chart.html.) to justify how you are filling in the missing values. (10)

----

In [ ]:
df = titanic.copy()
df = df[~df["Cabin"].isna()]
df["section"] = df["Cabin"].str.get(0)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,section
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,E
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,G
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,C


In [ ]:
titanic["Cabin"].str.get(0).unique()

array([nan, 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [ ]:
sections = []

for _ in range(3):
    for i in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']:
        sections.append(i)

classes = []
for i in ["1", "2", "3"]:
    for _ in range(8):
        classes.append(i)

dict_sections = {
    "section": sections,
    "pclass": classes,
    "number": [0]*24
}

ndf = pd.DataFrame(dict_sections)

grouped = df.groupby(["section", "Pclass"]).size()

for i in range(len(ndf)):
    try:
        ndf["number"][i] = grouped[ndf["section"][i]][int(ndf["pclass"][i])]
    except:
        # if those indexes don't exist
        pass


ndf["perc"] = [0]*24
for i in range(len(ndf)):
    total = ndf.loc[ndf["section"] == ndf["section"][i]]["number"].sum()
    
    ndf["perc"][i] = (((ndf["number"][i])/total)*100).round(0)

In [ ]:
fig = px.bar(ndf, 
    x="section", 
    y="perc", 
    color="pclass",
    text="number")
fig.show()

Therefore, if the passenger was of:

i) first class: we can place them in sections `A` `B` `C` `T` with very high confidence.

Food for thought: Would you choose `A` because more of them are in `A`? or `T`, because there is more space in `T`? Or would you fill them in the proportion of their probabiltities? While any of them is finally a guess, knowledge of the ship's blueprints, i.e., domain knowledge (this is just a google away :p) would help make this decision. 

Also, more data on these columns can help. Can you think of annother way to fill missing values of the cabins?

ii) second class: following a similar train of thought,`F` would be the best choice

iii) third class: `G`

What we've done here is **Feature Engineering** (of sorts, since we've created most of the values). We'll look into better examples of feature engineering in the next sections.

If you find this method *hinky*, but cannot think of any other method as well (whether it be finding information from this or other datasets), you can always drop the column.

----

#### Age

Again, like `Cabin`, `Age` isn't a categorical column, so we will need to think before just filling the missing value with a statistic like mode. 

Q3. Fill the missing values with any aggregate function, you think is suitable on the `Age` column of the dataframe grouped by the most correlated column (show correlation graphically). (20)

Hint: google "correlation heatmap" in the library of your choice.

In [ ]:
df = titanic.copy()
df["Pclass"] = df["Pclass"].astype(int)
px.imshow(df.corr(), text_auto=True)


`Pclass` is the most correlated with `Age` (-0.36), followed by `SibSp` (-0.30).

**Note:** There are generally 3 kinds of variables you'll come across: Continuous (like `Age`), Categorical (binary is a subset of categorical) (like `Survived`) and Ordinal(like `Pclass`).

Sometimes values in the correlation matrix cannot be taken directly due to types of these variables, understanding how to handle them in each case can be found in [this stackexchange question](https://stats.stackexchange.com/questions/369783/how-to-do-a-correlation-matrix-with-categorical-ordinal-and-interval-variable).

In our case, between ordinal and continuos variables, we can use the values in the correlation matrix directly.

In [ ]:
df[df["Age"].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


In [ ]:
grouped = df.groupby(["Pclass"])
age_series = grouped.mean()["Age"].round(0)

age_series

Pclass
1    38.0
2    30.0
3    25.0
Name: Age, dtype: float64

In [ ]:
grouped.median()["Age"].round(0)

Pclass
1    37.0
2    29.0
3    24.0
Name: Age, dtype: float64

In [ ]:
df.groupby(['Pclass'])['Age'].apply(pd.Series.mode)

Pclass   
1       0    35.0
        1    36.0
2       0    24.0
        1    34.0
3       0    22.0
Name: Age, dtype: float64

In [ ]:
fig = px.histogram(df, x = "Age", facet_col = "Pclass", category_orders = {"Pclass": [1, 2, 3]})
fig.show()

In case of `Pclass 1` we can see that the distribution is a near approximation of a uniform distribution with very few outliers, so we can use the mean value to fill the `NaN` values. 

In `Pclass 3`, we have unimodal distribution, but a skewed one, so filling the `NaN` values with it would be wrong. [Reason: Extra reading](https://vitalflux.com/pandas-impute-missing-values-mean-median-mode/). But we can fill them with either median or mode. (median is usually better in skewed distributions, and mode in those with more outliers)

In `Pclass 2`, we clearly have a distinct [bimodal distributuion](https://statisticsbyjim.com/basics/bimodal-distribution/), and we cannot fill those values directly.